<a href="https://colab.research.google.com/github/AsSugar13/ass13_M5/blob/master/new_begining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Base

In [0]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental                       import enable_iterative_imputer
from IPython.display                            import display

from sklearn.linear_model                       import LinearRegression, Ridge, Lasso, HuberRegressor, SGDRegressor
from sklearn.svm                                import SVC, NuSVR, OneClassSVM
from sklearn.pipeline                           import Pipeline, FeatureUnion
from sklearn.preprocessing                      import OrdinalEncoder, FunctionTransformer, OneHotEncoder,\
                                                       KBinsDiscretizer, LabelEncoder, PolynomialFeatures,\
                                                       StandardScaler, MaxAbsScaler, maxabs_scale, OrdinalEncoder
from imblearn.pipeline                          import Pipeline as Pipeline_IMB
from sklearn.impute                             import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics                            import make_scorer, mean_squared_error
from scipy.stats                                import spearmanr
from sklearn.model_selection                    import KFold, StratifiedKFold, GroupKFold, TimeSeriesSplit,\
                                                       RandomizedSearchCV, GridSearchCV
from sklearn.feature_extraction.text            import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_selection                  import SelectKBest, chi2, SelectFromModel, f_regression
from sklearn.decomposition                      import TruncatedSVD

from sklearn.linear_model                       import LogisticRegression
from sklearn.multioutput                        import MultiOutputClassifier, MultiOutputRegressor
from sklearn.multiclass                         import OneVsRestClassifier
from sklearn.ensemble                           import RandomForestClassifier, RandomForestRegressor,\
                                                       AdaBoostRegressor, AdaBoostClassifier, GradientBoostingRegressor
from sklearn.linear_model                       import ElasticNet, BayesianRidge
from sklearn.neural_network                     import MLPRegressor
import xgboost as xgb
from sklearn.model_selection                    import train_test_split, RandomizedSearchCV, GridSearchCV,\
                                                       cross_val_score, cross_val_predict, cross_validate

from imblearn                                   import FunctionSampler
from imblearn.over_sampling                     import SMOTE, SVMSMOTE, ADASYN
from imblearn.combine                           import SMOTETomek

import gc
import re
import pickle
sns.set(style="white", color_codes=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'
np.random.seed(13)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [0]:
### reading first lines

# with open('./sales_train_validation.csv', "r") as f:
#     for i in range(1):
#         print(f.readline())

In [0]:
def macd(df, slow):
    first, second, slow = slow*12//9, slow*26//9, slow
    
    first_ewm = df.ewm(span=first, min_periods=first, adjust=False, axis=1).mean().astype('float16')
    second_ewm = df.ewm(span=second, min_periods=second, adjust=False, axis=1).mean().astype('float16')
    
    fast_ema = first_ewm - second_ewm
    slow_ema = fast_ema.ewm(span=slow, min_periods=slow, adjust=False, axis=1).mean().astype('float16')
    
    macd = fast_ema-slow_ema
    
    return macd

def angle(x):
    return np.rad2deg(np.arctan(np.polyfit(range(len(x)), x, 1)[0])).astype('float16')

In [0]:
def load_df(def_path='./'):
    
    # Load dtypes
    with open(def_path + 'dtypes.pickle', 'rb') as f:
        dtypes = pickle.load(f)
        
    df = pd.read_csv(def_path + 'sales_train_validation.csv',
                     dtype=dtypes
#                  , index_col='id'
                    )
    gc.collect()
    return df

df = load_df(def_path='./drive/My Drive/Colab Notebooks/')

In [0]:
# display(df.shape)
# display(df.info())
# display(df.describe())
# display(df.head())

In [0]:
### define dtype for the 1st time

# dtypes = {}

# for i, j in zip(df.dtypes.index, df.dtypes):
#     if (j == 'int'):
# #         df.loc[:, i] = pd.to_numeric(df.loc[:, i], downcast='integer')
#         dtypes[i]='uint16'
#     elif (j == 'object'):
#         dtypes[i] = 'object'

# # saving dtype for future ease importing
# with open('dtypes.pickle', 'wb') as f:
#     pickle.dump(dtypes, f)

In [0]:
### changing dtypes for variables

# with open('./drive/My Drive/Colab Notebooks/dtypes.pickle', 'rb') as f:
#     dtypes = pickle.load(f)
# for i in dtypes.keys():
#     if dtypes[i] == 'uint16':
#         dtypes[i] = 'float16'
# with open('dtypes.pickle', 'wb') as f:
#     pickle.dump(dtypes, f)

In [0]:
nums = []
cats = []

for i, j in zip(df.dtypes.index, df.dtypes):
    if (j == 'int16'):
        nums.append(i)
    elif (j == 'object'):
        cats.append(i)

In [0]:
### melting data
df = df.melt(id_vars=cats, var_name='d', value_name='demand')

In [0]:
def adding_cols(df, lags=[0, 90, 180], macd_angle=False, by_col='id', target_col='demand'):
    
#     df = df.copy()
    
    params = []

    for lag in lags:
        print(lag)
    
        #rolling mean
        for window in [7, 14]:

            add_param = df.groupby(by=by_col)[target_col]\
            .transform(lambda x: x.rolling(window).mean()\
            .shift(1+lag)).astype('float16')
            
            sub_column_name = f'rol_mean_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()

        #rolling std
        for window in [7, 14]:
            
            add_param = df.groupby(by=by_col)[target_col]\
            .transform(lambda x: x.rolling(window).std()\
            .shift(1+lag)).astype('float16')
            
            sub_column_name = f'rol_std_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()
            
    #     #rolling max
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).max().astype('float16')
            
    #         sub_column_name = f'rol_max_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling min
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).min().astype('float16')
            
    #         sub_column_name = f'rol_min_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling skew
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).skew().astype('float16')
            
    #         sub_column_name = f'rol_skew_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling kurt
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).kurt().astype('float16')
            
    #         sub_column_name = f'rol_kurt_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
        #ewm mean    
        for window in [7, 14]:
            
            add_param = df.groupby(by=by_col)[target_col]\
            .transform(lambda x: x.ewm(window).mean()\
            .shift(1+lag)).astype('float16')
            
            sub_column_name = f'ewm_mean_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()
            
        #ewm std    
        for window in [7, 14]:
            
            add_param = df.groupby(by=by_col)[target_col]\
            .transform(lambda x: x.ewm(window).std()\
            .shift(1+lag)).astype('float16')
            
            sub_column_name = f'ewm_std_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()

    #     #macd
    #     for short in [4, 7, 9]:
    #         add_param = macd(df[nums], short)
    #         sub_column_name = f'macd_'+str(short)
    #         params.append(sub_column_name)
    #         add_param.columns = pd.MultiIndex.from_tuples([(col[0], sub_column_name) for col in add_param.columns])
    #         df_out = df_out.join(add_param).sort_index(axis=1)
    #         gc.collect()

    #         if macd_angle==True:
    #             for window in [2, 3, 5]:
    #                 add_param = add_param.rolling(window, axis=1).apply(angle, raw=True)
    #                 sub_column_name = f'macd_'+str(short)+'_angle_'+str(window)
    #                 params.append(sub_column_name)
    #                 add_param.columns = pd.MultiIndex.from_tuples([(col[0], sub_column_name) for col in add_param.columns])
    #                 df_out = df_out.join(add_param).sort_index(axis=1)
    #                 gc.collect()

    # gc.collect()
    return df, params

In [0]:
def load_calendar(def_path='./'):  
    calendar = pd.read_csv(def_path + 'calendar.csv',
                           
                           usecols=['wday', 'month', 'event_name_1', 'event_type_1',
                                    'event_name_2', 'event_type_2',
                                    'snap_CA', 'snap_TX', 'snap_WI', 'd'],

                           dtype={'wday': 'uint8',
                                  'month': 'uint8',
                                  'snap_CA': 'uint8',
                                  'snap_TX': 'uint8',
                                  'snap_WI': 'uint8',                            
                                 }
                          )
    
    calendar.fillna('Nothing', inplace=True)
    # calendar.drop(['date', 'wm_yr_wk', 'weekday','year'])
    
    cats_to_encode = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    
#####################################################################################################    
    event_name_1_in_1d =(calendar['event_name_1'].shift(-1, axis=0).fillna('Nothing')!='Nothing')
    event_name_1_in_1d.name = 'event_name_1_in_1d'
    cats_to_encode.append('event_name_1_in_1d')
    
    event_name_1_in_3d =(calendar['event_name_1'].shift(-1, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-2, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-3, axis=0).fillna('Nothing')!='Nothing')
    event_name_1_in_3d.name = 'event_name_1_in_3d'
    cats_to_encode.append('event_name_1_in_3d')

    event_name_1_in_7d =(calendar['event_name_1'].shift(-1, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-2, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-3, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-4, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-5, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-6, axis=0).fillna('Nothing')!='Nothing')|\
                        (calendar['event_name_1'].shift(-7, axis=0).fillna('Nothing')!='Nothing')
    event_name_1_in_7d.name = 'event_name_1_in_7d'
    cats_to_encode.append('event_name_1_in_7d')
    
    calendar = pd.concat([calendar, event_name_1_in_1d, event_name_1_in_3d, event_name_1_in_7d], axis=1)
#####################################################################################################

#params
    params = calendar.columns.tolist()

#encoding  cats
    encoder = OrdinalEncoder(dtype='uint8')    
    calendar.loc[:, cats_to_encode] = encoder.fit_transform(calendar[cats_to_encode])
    for i in cats_to_encode:
        calendar.loc[:, i] = calendar.loc[:, i].astype('uint8')
    
    return calendar, params

In [0]:
%%time
df.loc[:, 'demand_log'] = df['demand'].apply(lambda x: np.log1p(x)).astype('float16')
df, params_df = adding_cols(df, lags=[0, 30, 180], target_col='demand_log')
calendar, params_calendar = load_calendar(def_path='./drive/My Drive/Colab Notebooks/')

df = df.merge(calendar, how='left', on='d')
del calendar

df.dropna(axis=0, inplace=True)
print('Completed')

In [0]:
# %%time
# df, params_df = adding_cols(df, lags=[0, 30, 180])
# calendar, params_calendar = load_calendar(def_path='./drive/My Drive/Colab Notebooks/')

# df = df.merge(calendar, how='left', on='d')
# del calendar

# df.dropna(axis=0, inplace=True)
# print('Completed')

0
30
180
Completed
CPU times: user 22min 22s, sys: 12.1 s, total: 22min 34s
Wall time: 22min 36s


# Pytorch Init

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

import torch
print(torch.cuda.is_available())

# print(!lscpu)

Sun May  3 21:59:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
display(df.head())

display(df.id.value_counts().shape)
display(df.item_id.value_counts().shape)
display(df.dept_id.value_counts().shape)
display(df.cat_id.value_counts().shape)
display(df.store_id.value_counts().shape)
display(df.state_id.value_counts().shape)
display(df.d.value_counts().shape)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,rol_mean_t7_l0,rol_mean_t14_l0,rol_std_t7_l0,rol_std_t14_l0,ewm_mean_t7_l0,ewm_mean_t14_l0,ewm_std_t7_l0,ewm_std_t14_l0,rol_mean_t7_l30,rol_mean_t14_l30,rol_std_t7_l30,rol_std_t14_l30,ewm_mean_t7_l30,ewm_mean_t14_l30,ewm_std_t7_l30,ewm_std_t14_l30,rol_mean_t7_l180,rol_mean_t14_l180,rol_std_t7_l180,rol_std_t14_l180,ewm_mean_t7_l180,ewm_mean_t14_l180,ewm_std_t7_l180,ewm_std_t14_l180,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_name_1_in_1d,event_name_1_in_3d,event_name_1_in_7d
5915060,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_195,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,8,19,2,3,1,0,1,1,0,0,0
5915061,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_195,0,0.285645,0.142822,0.488037,0.363037,0.157593,0.151123,0.377197,0.364502,0.142822,0.285645,0.377930,0.468750,0.209961,0.194702,0.421631,0.402832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,8,19,2,3,1,0,1,1,0,0,0
5915062,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_195,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,8,19,2,3,1,0,1,1,0,0,0
5915063,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_195,0,1.286133,1.786133,1.112305,1.528320,1.422852,1.542969,1.405273,1.472656,2.142578,2.000000,1.772461,1.753906,2.039062,1.716797,1.822266,1.758789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,8,19,2,3,1,0,1,1,0,0,0
5915064,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_195,0,0.285645,0.714355,0.755859,1.204102,0.553223,0.482910,1.042969,1.035156,0.000000,0.071411,0.000000,0.267334,0.178711,0.442871,0.812012,1.246094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,8,19,2,3,1,0,1,1,0,0,0


(30490,)

(3049,)

(7,)

(3,)

(10,)

(3,)

(1719,)

In [0]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [0]:
def emb_init(x):
    x = x.weight.data
    std = 2/(x.size(1)+1)
    torch.nn.init.normal_(x, mean=0, std=std)
    # sc = 2/(x.size(1)+1)
    # x.uniform_(-sc,sc)

In [0]:
class LSTMModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, ninp, batch_size, nlayers=8, dropout=0.2):
        super(LSTMModel, self).__init__()


        self.dropout = nn.Dropout(dropout)
        self.rnn1 = nn.LSTM(ninp, 32, nlayers, dropout=dropout)
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 1)
        # self.fc3 = nn.Linear(8, 1)

        self.batch_size = batch_size
        self.ninp = ninp

    def forward(self, x):

        x = x.view(len(x), 1, -1)
        x, hid = self.rnn1(x)
        x = self.dropout(self.fc1(x).sum(1))
        # x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        # x = self.dropout(self.fc3(x))

        return x

In [0]:
class Embedding(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""
    
    def __init__(self, emb_szs, batch_size=64, dropout=0.2):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        self.batch_size = batch_size

        self.embeddings = nn.ModuleList([nn.Embedding(r, c) for r, c in emb_szs])
        for i in self.embeddings: emb_init(i)

        emb_len = sum(i.embedding_dim for i in self.embeddings)

        self.fc1 = nn.Linear(emb_len, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = [emb(x[:, i].long()) for i, emb in enumerate(self.embeddings)]
        x = torch.cat(x, dim=1)
        # x = self.drop(x)

        x = self.fc1(x)
        # x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        # x = self.dropout(self.fc3(x))

        return x

In [0]:
class LSTM_Emb_Model(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, emb_szs, cont_len, nlayers=8, dropout=0.2):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(cont_len)

        self.embeddings = nn.ModuleList([nn.Embedding(r, c) for r, c in emb_szs])
        for i in self.embeddings: emb_init(i)
        emb_len = sum(i.embedding_dim for i in self.embeddings)
        ninp = emb_len + cont_len

        self.rnn1 = nn.LSTM(ninp, 8, nlayers, dropout=dropout)
        # self.fc0 = nn.Linear(ninp, 64)

        self.fc1 = nn.Linear(8, 1)
        # self.fc2 = nn.Linear(16, 1)
        # self.fc3 = nn.Linear(16, 1)

    def forward(self, x_cont, x_cat):

        x_cat = [emb(x_cat[:, i].long()) for i, emb in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        x_cont = self.bn(x_cont)
        x = self.dropout(torch.cat([x_cont, x_cat], dim=1))
        x = x.view(len(x), 1, -1)

        x, hid = self.rnn1(x)
        x = self.dropout(self.fc1(x).sum(1))

        # x = self.dropout(self.fc0(x))
        # x = self.dropout(self.fc1(x))
        # x = self.dropout(self.fc2(x))


        return x

In [0]:
class LSTM_Emb_Model_2(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, emb_szs, cont_len, nlayers=7, dropout=0.2):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(cont_len)

        self.embeddings = nn.ModuleList([nn.Embedding(r, c) for r, c in emb_szs])
        for i in self.embeddings: emb_init(i)
        emb_len = sum(i.embedding_dim for i in self.embeddings)
        ninp = emb_len + cont_len

        self.rnn1 = nn.LSTM(cont_len, 14, nlayers, dropout=dropout, bidirectional=True)
        # self.fc_cats = nn.Linear(emb_len, 1)
        self.fc1 = nn.Linear(emb_len+28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x_cont, x_cat):

        x_cat = [emb(x_cat[:, i].long()) for i, emb in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        # x_cat = self.dropout(self.fc_cats(x_cat))

        x_cont = self.bn(x_cont)
        x_cont = x_cont.view(len(x_cont), 1, -1)
        x_cont, hid = self.rnn1(x_cont)
        x_cont = x_cont.sum(1)

        x = self.dropout(torch.cat([x_cont, x_cat], dim=1))
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

In [0]:
class RNN_Emb_Model(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, emb_szs, cont_len, nlayers=7, dropout=0.2):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(cont_len)

        self.embeddings = nn.ModuleList([nn.Embedding(r, c) for r, c in emb_szs])
        for i in self.embeddings: emb_init(i)
        emb_len = sum(i.embedding_dim for i in self.embeddings)
        ninp = emb_len + cont_len

        self.rnn1 = nn.RNN(cont_len, 30, nlayers, dropout=dropout, bidirectional=True)
        # self.fc_cats = nn.Linear(emb_len, 1)
        self.fc1 = nn.Linear(emb_len+30*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x_cont, x_cat):

        x_cat = [emb(x_cat[:, i].long()) for i, emb in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        # x_cat = self.dropout(self.fc_cats(x_cat))

        x_cont = self.bn(x_cont)
        x_cont = x_cont.view(len(x_cont), 1, -1)
        x_cont, hid = self.rnn1(x_cont)
        x_cont = x_cont.sum(1)

        x = self.dropout(torch.cat([x_cont, x_cat], dim=1))
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

In [0]:
class RNN_Emb_Exp_Model(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, emb_szs, cont_len, nlayers=7, dropout=0.2):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(cont_len)

        self.embeddings = nn.ModuleList([nn.Embedding(r, c) for r, c in emb_szs])
        for i in self.embeddings: emb_init(i)
        emb_len = sum(i.embedding_dim for i in self.embeddings)
        ninp = emb_len + cont_len

        self.rnn1 = nn.RNN(cont_len, 30, nlayers, dropout=dropout, bidirectional=True)
        # self.fc_cats = nn.Linear(emb_len, 1)
        self.fc1 = nn.Linear(emb_len+30*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x_cont, x_cat):

        x_cat = [emb(x_cat[:, i].long()) for i, emb in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        # x_cat = self.dropout(self.fc_cats(x_cat))

        # x_cont = torch.log1p(x_cont)
        x_cont = self.bn(x_cont)
        x_cont = x_cont.view(len(x_cont), 1, -1)
        x_cont, hid = self.rnn1(x_cont)
        x_cont = x_cont.sum(1)
        # x_cont = torch.expm1(x_cont)

        x = self.dropout(torch.cat([x_cont, x_cat], dim=1))
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

#Checkpoint

#Data Pipeline

In [0]:
nums = params_df
cats = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 'month', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
bools = ['snap_CA', 'snap_TX', 'snap_WI', 'event_name_1_in_1d', 'event_name_1_in_3d', 'event_name_1_in_7d']

get_cat = FunctionTransformer(lambda df: df.loc[:, cats], validate=False)
get_num = FunctionTransformer(lambda df: df.loc[:, nums], validate=False)
get_bool = FunctionTransformer(lambda df: df.loc[:, bools], validate=False)

In [0]:
#masks for nn eval
test_edge_start = 1886 #1913-28+1
test_edge_end = 1913 #1913

test_mask_start_ind = df[df.d == 'd_'+str(test_edge_start)].index[0]
test_mask_end_ind = df[df.d == 'd_'+str(test_edge_end)].index[-1]
# test_mask = df.loc[test_mask_start_ind:test_mask_end_ind, :]

val_edge_start = 1856 #1886-30+1
val_edge_end = 1885 #1913-28+1

val_mask_start_ind = df[df.d == 'd_'+str(val_edge_start)].index[0]
val_mask_end_ind = df[df.d == 'd_'+str(val_edge_end)].index[-1]
# val_mask = df.loc[val_mask_start_ind:val_mask_end_ind, :]

# train_edge_start = 1095
train_edge_start = 195
# train_edge_start = 1676
# train_edge_start = 1126
train_edge_end = 1855

train_mask_start_ind = df[df.d == 'd_'+str(train_edge_start)].index[0]
train_mask_end_ind = df[df.d == 'd_'+str(train_edge_end)].index[-1]

In [0]:
X_test = df.loc[test_mask_start_ind:test_mask_end_ind, nums+cats+bools]
y_test = df.loc[test_mask_start_ind:test_mask_end_ind, :]['demand']

display(X_test.shape)
display(y_test.shape)

X_val = df.loc[val_mask_start_ind:val_mask_end_ind, nums+cats+bools]
y_val = df.loc[val_mask_start_ind:val_mask_end_ind, :]['demand']

display(X_val.shape)
display(y_val.shape)

X = df.loc[train_mask_start_ind:train_mask_end_ind, nums+cats+bools]
y = df.loc[train_mask_start_ind:train_mask_end_ind, :]['demand']

display(X.shape)
display(y.shape)

# display(df.shape)
# del df

(853720, 42)

(853720,)

(914700, 42)

(914700,)

(50643890, 42)

(50643890,)

In [0]:
# Pipeline obviously
pl = Pipeline_IMB([    
    ('union_outer', FeatureUnion(transformer_list=[
#         ('get_nums', Pipeline([
#             ('get_num', get_num),
# #             ('discretizer', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')),
#         ])),
        ('get_cat', Pipeline([
            ('get_cat', get_cat),
            ('lab_enc', OrdinalEncoder())
            # ('ohe', OneHotEncoder(dtype='uint8', handle_unknown='ignore'))
        ])),
        ('get_bools', Pipeline([
            ('get_num', get_bool),
#             ('discretizer', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')),
        ])),
    ], n_jobs=1, verbose=True)),
    
#     ('dim_red', SelectKBest(f_regression, 5000)),
#     ('sampling', SMOTE(sampling_strategy='auto', n_jobs=12)),
#     ('decomp', TruncatedSVD(n_components=100)),
#     ('dim_red', SelectKBest(chi2, 10000)),
#     ('imp', SimpleImputer(strategy="median")),
#     ('inter', SparseInteractions(degree=2)),
#     ('inter', PolynomialFeatures(degree=2, order='F')),
    # ('scale', MaxAbsScaler()),
    # ('scale', StandardScaler()),
#     ('sfm', SelectFromModel(xgb.XGBRegressor(max_depth=100, tree_method='approx', n_jobs=16), threshold='0.5*mean')),
#     ('dim_red', SelectKBest(chi2, 50)),
#     ('clf', LinearRegression(n_jobs=-1))
#     ('clf', LogisticRegression(class_weight='balanced', n_jobs=12))
#     ('sfm', SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=3), threshold=0.015)),
#     ('clf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=7))
#     ('sfm', SelectFromModel(xgb.XGBRFRegressor(tree_method='hist', random_state=42, n_jobs=12), threshold=0.015)),
#     ('xgb', MultiOutputRegressor(xgb.XGBRFRegressor(objective='reg:squarederror', max_depth=45, tree_method='hist', n_jobs=12,random_state=42)))
#     ('xgb', xgb.XGBRegressor(max_depth=100, tree_method='approx', n_jobs=7))
#     ('clf', LogisticRegression(class_weight='balanced', n_jobs=12))
#     ('clf', RandomForestClassifier(class_weight='balanced', n_jobs=12))
    # ('svc', NuSVR(degree=3, kernel='poly'))
#     ('nn', MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100)))
#     ('svc', SVC(class_weight='balanced'))
#     ('clf', GradientBoostingRegressor(loss= 'huber', max_depth=10))
#     ('clf', MLPRegressor())
])

In [0]:
def cat_encoder(pandas_df, cat_cols_list, dicts=None):
    if dicts == None: dicts_ = []
    
    for i, col in enumerate(cat_cols_list):
        if dicts == None:
            dict_ = {o:i for i, o in enumerate(pandas_df[col].unique())}
            dicts_.append(dict_)
        else: dict_ = dicts[i]

        pandas_df.loc[:, col] = pandas_df[col].apply(lambda x: dict_[x]).astype('uint16')
    
    return dicts



In [0]:
%%time
dicts = cat_encoder(X, cats)
cat_encoder(X_val, cats, dicts=dicts)
cat_encoder(X_test, cats, dicts=dicts)

CPU times: user 9min 54s, sys: 1.04 s, total: 9min 56s
Wall time: 9min 55s


In [0]:
# X_torch = pl.fit_transform(X)
# X_val_torch = pl.transform(X_val)
# X_test_torch = pl.transform(X_test)

# X_torch = X
# X_val_torch = X_val
# X_test_torch = X_test

In [0]:
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 30490

##### NUMS
try:
    X_nums_t = torch.tensor(X[nums].values)
    X_nums_val_t = torch.tensor(X_val[nums].values, requires_grad=False)
    X_nums_test_t = torch.tensor(X_test[nums].values, requires_grad=False)
except:
    X_nums_t = torch.tensor(X[nums].values.astype('float32'))
    X_nums_val_t = torch.tensor(X_val[nums].values.astype('float32'), requires_grad=False)
    X_nums_test_t = torch.tensor(X_test[nums].values.astype('float32'), requires_grad=False)
##### CATS & BOOLS
try:
    X_cats_t = torch.tensor(X[cats+bools].values)
    X_cats_val_t = torch.tensor(X_val[cats+bools].values, requires_grad=False)
    X_cats_test_t = torch.tensor(X_test[cats+bools].values, requires_grad=False)
except:
    X_cats_t = torch.tensor(X[cats+bools].values.astype('float32'))
    X_cats_val_t = torch.tensor(X_val[cats+bools].values.astype('float32'), requires_grad=False)
    X_cats_test_t = torch.tensor(X_test[cats+bools].values.astype('float32'), requires_grad=False)

y_torch = torch.tensor(y.values.astype('float32')).view(-1, 1)
y_val_torch = torch.tensor(y_val.values.astype('float32')).view(-1, 1)
y_test_torch = torch.tensor(y_test.values.astype('float32')).view(-1, 1)

trainloader = DataLoader(TensorDataset(X_nums_t, X_cats_t, y_torch), batch_size=batch_size)
validloader = DataLoader(TensorDataset(X_nums_val_t, X_cats_val_t, y_val_torch), batch_size=batch_size)
testloader = DataLoader(TensorDataset(X_nums_test_t, X_cats_test_t, y_test_torch), batch_size=batch_size)

In [0]:
# dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
# for i, data in enumerate(dataloader):
#     data['image'] = data['image'].type(dtype)
#     data['label'] = data['label'].type(dtype)

In [0]:
%%time
def szs_def(df, columns_list):
    szs = []
    for col in columns_list:
        unique = df[col].nunique()
        dim_fun = lambda x: int(x) if (x <= 10) else int(np.sqrt(x))
        szs.append([unique, dim_fun(unique)])
    return szs

szs = szs_def(df, df[cats+bools].columns)

CPU times: user 19 s, sys: 934 ms, total: 19.9 s
Wall time: 19.9 s


# Model Training

In [0]:
# def QLL(predicted, observed):
#     p = torch.tensor(1.5)
#     QLL = QLL = torch.pow(predicted, (-p))*(((predicted*observed)/(1-p)) - ((torch.pow(predicted, 2))/(2-p)))

#     return QLL
        
# def tweedieloss(predicted, observed):
#     '''
#     Custom loss fuction designed to minimize the deviance using stochastic gradient descent
#     tweedie deviance from McCullagh 1983

#     '''
#     d = -2*QLL(predicted, observed)
# #     loss = (weight*d)/1


#     return torch.mean(d)

In [0]:
# device = torch.device("cpu")

model = RNN_Emb_Model(emb_szs=szs, cont_len=len(nums))

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)
model.train()

RNN_Emb_Model(
  (dropout): Dropout(p=0.2, inplace=False)
  (bn): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (embeddings): ModuleList(
    (0): Embedding(30490, 174)
    (1): Embedding(3049, 55)
    (2): Embedding(7, 7)
    (3): Embedding(3, 3)
    (4): Embedding(10, 10)
    (5): Embedding(3, 3)
    (6): Embedding(7, 7)
    (7): Embedding(12, 3)
    (8): Embedding(31, 5)
    (9): Embedding(5, 5)
    (10): Embedding(4, 4)
    (11): Embedding(3, 3)
    (12): Embedding(2, 2)
    (13): Embedding(2, 2)
    (14): Embedding(2, 2)
    (15): Embedding(2, 2)
    (16): Embedding(2, 2)
    (17): Embedding(2, 2)
  )
  (rnn1): LSTM(24, 30, num_layers=7, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=351, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [0]:
%%time
epochs = 1
running_loss = 0
print_every = 15
for epoch in range(epochs):
        for batch_ind, (inputs_cont, inputs_cat, labels) in enumerate(trainloader):

            # Move input and label tensors to the default device/
            inputs_cont, inputs_cat, labels = inputs_cont.to(device), inputs_cat.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs_cont.float(), inputs_cat.float())
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            gc.collect()

            if (batch_ind+1) % print_every == 0:
                valid_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs_cont, inputs_cat, labels in validloader:
                        inputs_cont, inputs_cat, labels = inputs_cont.to(device), inputs_cat.to(device), labels.to(device)

                        logps = model.forward(inputs_cont.float(), inputs_cat.float())
                        batch_loss = criterion(logps, labels)

                        valid_loss += batch_loss.item()

                        # Calculate accuracy
                        # ps = torch.exp(logps)
                        # top_p, top_class = ps.topk(1, dim=1)
                        # equals = top_class == labels.view(*top_class.shape)
                        # accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Valid loss: {valid_loss/len(validloader):.3f}.. "
                    #   f"Valid accuracy: {accuracy/len(validloader):.3f}"
                    )
                # gpu_info = !nvidia-smi
                # gpu_info = '\n'.join(gpu_info)
                # print(gpu_info)
                model.train()
                running_loss = 0

Epoch 1/1.. Train loss: 14.589.. Valid loss: 15.067.. 
Epoch 1/1.. Train loss: 12.737.. Valid loss: 14.818.. 
Epoch 1/1.. Train loss: 12.526.. Valid loss: 14.481.. 
Epoch 1/1.. Train loss: 18.987.. Valid loss: 14.072.. 
Epoch 1/1.. Train loss: 16.421.. Valid loss: 13.693.. 
Epoch 1/1.. Train loss: 15.398.. Valid loss: 13.483.. 
Epoch 1/1.. Train loss: 18.198.. Valid loss: 13.268.. 
Epoch 1/1.. Train loss: 14.639.. Valid loss: 13.286.. 
Epoch 1/1.. Train loss: 18.545.. Valid loss: 12.917.. 
Epoch 1/1.. Train loss: 17.611.. Valid loss: 12.454.. 
Epoch 1/1.. Train loss: 14.009.. Valid loss: 12.176.. 
Epoch 1/1.. Train loss: 13.351.. Valid loss: 12.273.. 
Epoch 1/1.. Train loss: 13.755.. Valid loss: 11.995.. 
Epoch 1/1.. Train loss: 12.024.. Valid loss: 12.280.. 
Epoch 1/1.. Train loss: 11.951.. Valid loss: 11.707.. 
Epoch 1/1.. Train loss: 11.249.. Valid loss: 12.102.. 
Epoch 1/1.. Train loss: 11.907.. Valid loss: 11.254.. 
Epoch 1/1.. Train loss: 10.060.. Valid loss: 11.633.. 
Epoch 1/1.

In [0]:
torch.save(model, './drive/My Drive/Colab Notebooks/model_LSTM30_Emb')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNN_Emb_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
# RNN bidir hid=30
Epoch 1/1.. Train loss: 5.477.. Valid loss: 5.005.. 
Epoch 1/1.. Train loss: 5.027.. Valid loss: 4.953.. 
Epoch 1/1.. Train loss: 5.269.. Valid loss: 4.812.. 
Epoch 1/1.. Train loss: 4.136.. Valid loss: 4.818.. 
Epoch 1/1.. Train loss: 5.560.. Valid loss: 4.782.. 

# LSTM bidir hid=30
Epoch 1/1.. Train loss: 5.935.. Valid loss: 5.693.. 
Epoch 1/1.. Train loss: 5.516.. Valid loss: 5.541.. 
Epoch 1/1.. Train loss: 5.644.. Valid loss: 5.476.. 
Epoch 1/1.. Train loss: 4.432.. Valid loss: 5.446.. 
Epoch 1/1.. Train loss: 5.879.. Valid loss: 5.406.. 

In [0]:
# RNN hid=30
Epoch 1/5.. Train loss: 2.764.. Valid loss: 1.942.. 
Epoch 1/5.. Train loss: 2.572.. Valid loss: 1.782.. 
Epoch 1/5.. Train loss: 2.842.. Valid loss: 1.937.. 
Epoch 1/5.. Train loss: 2.340.. Valid loss: 1.724.. 
Epoch 1/5.. Train loss: 2.934.. Valid loss: 1.726..

# RNN bidir hid=30
Epoch 1/1.. Train loss: 1.418.. Valid loss: 0.749.. 
Epoch 1/1.. Train loss: 1.487.. Valid loss: 0.735.. 
Epoch 1/1.. Train loss: 1.530.. Valid loss: 0.754.. 
Epoch 1/1.. Train loss: 1.352.. Valid loss: 0.722.. 
Epoch 1/1.. Train loss: 1.618.. Valid loss: 0.687.. 

# RNN bidir hid=30 Exp1(Log1(X))
Epoch 1/1.. Train loss: 1.627.. Valid loss: 0.927.. 
Epoch 1/1.. Train loss: 1.767.. Valid loss: 1.041.. 
Epoch 1/1.. Train loss: 1.779.. Valid loss: 0.882.. 
Epoch 1/1.. Train loss: 1.528.. Valid loss: 0.900.. 
Epoch 1/1.. Train loss: 1.877.. Valid loss: 0.921.. 

# RNN bidir hid=60
Epoch 1/1.. Train loss: 0.942.. Valid loss: 0.331.. 
Epoch 1/1.. Train loss: 1.068.. Valid loss: 0.541.. 
Epoch 1/1.. Train loss: 1.153.. Valid loss: 0.342.. 
Epoch 1/1.. Train loss: 1.049.. Valid loss: 0.612.. 
Epoch 1/1.. Train loss: 1.120.. Valid loss: 0.375..  

# RNN bidir hid=30 Log1(X) Exp1(y)
Epoch 1/2.. Train loss: 2.888.. Valid loss: 1.862.. 
Epoch 1/2.. Train loss: 2.647.. Valid loss: 1.921.. 
Epoch 1/2.. Train loss: 3.305.. Valid loss: 1.624.. 
Epoch 1/2.. Train loss: 2.435.. Valid loss: 1.664.. 
Epoch 1/2.. Train loss: 3.482.. Valid loss: 1.796..

# RNN bidir hid=90
Epoch 1/1.. Train loss: 0.795.. Valid loss: 0.255.. 
Epoch 1/1.. Train loss: 0.992.. Valid loss: 0.407.. 
Epoch 1/1.. Train loss: 0.994.. Valid loss: 0.261.. 
Epoch 1/1.. Train loss: 0.885.. Valid loss: 0.349.. 
Epoch 1/1.. Train loss: 0.917.. Valid loss: 0.391.. 

#Loaded models work

In [0]:
model = torch.load('./drive/My Drive/Colab Notebooks/model_RNN90_Emb')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.RNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch

In [0]:
# display(df.head())

template = df[df.d=='d_180'].iloc[:,:7].copy() # делаю шаблон на 30490 строк

In [0]:
df_ = df.copy()
d = 'd_1914'
template.loc[:, 'd'] = d
display(template)

,id,item_id,dept_id,cat_id,store_id,state_id,d
5457710,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914
5457711,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914
5457712,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914
5457713,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914
5457714,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914
...,...,...,...,...,...,...,...
5488195,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1914
5488196,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1914
5488197,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1914
5488198,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1914


In [0]:
df.loc[58325931:58325933,:]

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,rol_mean_t7_l0,rol_mean_t14_l0,rol_std_t7_l0,rol_std_t14_l0,ewm_mean_t7_l0,ewm_mean_t14_l0,ewm_std_t7_l0,ewm_std_t14_l0,rol_mean_t7_l30,rol_mean_t14_l30,rol_std_t7_l30,rol_std_t14_l30,ewm_mean_t7_l30,ewm_mean_t14_l30,ewm_std_t7_l30,ewm_std_t14_l30,rol_mean_t7_l180,rol_mean_t14_l180,rol_std_t7_l180,rol_std_t14_l180,ewm_mean_t7_l180,ewm_mean_t14_l180,ewm_std_t7_l180,ewm_std_t14_l180,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_name_1_in_1d,event_name_1_in_3d,event_name_1_in_7d
58325931,HOUSEHOLD_2_515_WI_3_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,0.142822,0.071411,0.37793,0.267334,0.114929,0.169678,0.399902,0.48291,0.285645,0.142822,0.488037,0.363037,0.242310,0.254639,0.563965,0.662598,0.285645,0.142822,0.755859,0.534668,0.208008,0.168945,0.619141,0.525391,2,4,19,2,3,1,0,0,0,0,0,1
58325932,HOUSEHOLD_2_516_WI_3_validation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1913,0,0.000000,0.071411,0.00000,0.267334,0.100586,0.158447,0.376221,0.46875,0.142822,0.142822,0.377930,0.363037,0.212036,0.237671,0.534180,0.643555,0.285645,0.142822,0.755859,0.534668,0.182007,0.157715,0.583496,0.509277,2,4,19,2,3,1,0,0,0,0,0,1
58325933,FOODS_1_001_WI_3_validation,FOODS_1_001,FOODS_1,FOODS,WI_3,WI,d_1913,0,0.000000,0.071411,0.00000,0.267334,0.088013,0.147827,0.353516,0.45459,0.142822,0.142822,0.377930,0.363037,0.185547,0.221802,0.504883,0.624512,0.428467,0.214233,0.786621,0.579102,0.284180,0.213867,0.613281,0.536621,2,4,19,2,3,1,0,0,0,0,0,1


In [0]:
df.groupby(by='item_id')['demand']\
.transform(lambda x: x.shift(0))\
.rolling(7).mean().astype('float16')

548833           NaN
548834           NaN
548835           NaN
548836           NaN
548837           NaN
              ...   
58327365    1.000000
58327366    0.856934
58327367    0.714355
58327368    0.856934
58327369    0.714355
Name: demand, Length: 57778537, dtype: float16

In [0]:
test = df.groupby(by='id')['demand']\
.rolling(7).mean().astype('float16')

idx = pd.IndexSlice
test.loc[idx['FOODS_3_827_CA_1_validation', :]]


# test.loc[slice('FOODS_3_827_CA_1_validation'), slice(None)]

id                                   
FOODS_3_827_CA_1_validation  551868           NaN
                             582358           NaN
                             612848           NaN
                             643338           NaN
                             673828           NaN
                                           ...   
                             58177968    7.714844
                             58208458    7.000000
                             58238948    7.000000
                             58269438    6.714844
                             58299928    7.000000
Name: demand, Length: 1895, dtype: float16

In [0]:
test.index

MultiIndex([(    'FOODS_1_001_CA_1_validation',   550432),
            (    'FOODS_1_001_CA_1_validation',   580922),
            (    'FOODS_1_001_CA_1_validation',   611412),
            (    'FOODS_1_001_CA_1_validation',   641902),
            (    'FOODS_1_001_CA_1_validation',   672392),
            (    'FOODS_1_001_CA_1_validation',   702882),
            (    'FOODS_1_001_CA_1_validation',   733372),
            (    'FOODS_1_001_CA_1_validation',   763862),
            (    'FOODS_1_001_CA_1_validation',   794352),
            (    'FOODS_1_001_CA_1_validation',   824842),
            ...
            ('HOUSEHOLD_2_516_WI_3_validation', 58051522),
            ('HOUSEHOLD_2_516_WI_3_validation', 58082012),
            ('HOUSEHOLD_2_516_WI_3_validation', 58112502),
            ('HOUSEHOLD_2_516_WI_3_validation', 58142992),
            ('HOUSEHOLD_2_516_WI_3_validation', 58173482),
            ('HOUSEHOLD_2_516_WI_3_validation', 58203972),
            ('HOUSEHOLD_2_516_WI_3_valid

In [0]:
df[df.item_id=='FOODS_3_827']['demand']\
.transform(lambda x: x.shift(0))\
.rolling(7).mean().astype('float16')

551868           NaN
554917           NaN
557966           NaN
561015           NaN
564064           NaN
              ...   
58315173    6.000000
58318222    6.429688
58321271    3.427734
58324320    3.714844
58327369    1.857422
Name: demand, Length: 18950, dtype: float16

In [0]:
df[df.item_id=='FOODS_3_826']['demand']\
.transform(lambda x: x.shift(0))\
.rolling(7).mean().astype('float16')

551867           NaN
554916           NaN
557965           NaN
561014           NaN
564063           NaN
              ...   
58315172    1.713867
58318221    1.857422
58321270    2.142578
58324319    1.857422
58327368    1.857422
Name: demand, Length: 18950, dtype: float16

In [0]:
df_ = pd.concat([df_, template], axis=0, ignore_index=True)

In [0]:
df__

In [0]:
def adding_cols(df, lags=[0, 90, 180], macd_angle=False, target_col='demand'):
    
#     df = df.copy()
    
    params = []

    for lag in lags:
        print(lag)
    
        #rolling mean
        for window in [7, 14]:

            add_param = df.groupby(by='item_id')[target_col]\
            .transform(lambda x: x.shift(lag))\
            .rolling(window).mean().astype('float16')
            
            sub_column_name = f'rol_mean_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()

        #rolling std
        for window in [7, 14]:
            
            add_param = df.groupby(by='item_id')[target_col]\
            .transform(lambda x: x.shift(lag))\
            .rolling(window).std().astype('float16')
            
            sub_column_name = f'rol_std_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()
            
    #     #rolling max
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).max().astype('float16')
            
    #         sub_column_name = f'rol_max_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling min
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).min().astype('float16')
            
    #         sub_column_name = f'rol_min_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling skew
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).skew().astype('float16')
            
    #         sub_column_name = f'rol_skew_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
    #     #rolling kurt
    #     for window in [7, 14, 30]:
            
    #         add_param = df.groupby(by='item_id')[target_col]\
    #         .transform(lambda x: x.shift(lag))\
    #         .rolling(window).kurt().astype('float16')
            
    #         sub_column_name = f'rol_kurt_t{window}_l{lag}'
    #         params.append(sub_column_name)
    #         df.loc[:, sub_column_name] = add_param
    #         gc.collect()
            
        #ewm mean    
        for window in [7, 14]:
            
            add_param = df.groupby(by='item_id')[target_col]\
            .transform(lambda x: x.shift(lag))\
            .ewm(window).mean().astype('float16')
            
            sub_column_name = f'ewm_mean_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()
            
        #ewm std    
        for window in [7, 14]:
            
            add_param = df.groupby(by='item_id')[target_col]\
            .transform(lambda x: x.shift(lag))\
            .ewm(window).std().astype('float16')
            
            sub_column_name = f'ewm_std_t{window}_l{lag}'
            params.append(sub_column_name)
            df.loc[:, sub_column_name] = add_param
            gc.collect()

    #     #macd
    #     for short in [4, 7, 9]:
    #         add_param = macd(df[nums], short)
    #         sub_column_name = f'macd_'+str(short)
    #         params.append(sub_column_name)
    #         add_param.columns = pd.MultiIndex.from_tuples([(col[0], sub_column_name) for col in add_param.columns])
    #         df_out = df_out.join(add_param).sort_index(axis=1)
    #         gc.collect()

    #         if macd_angle==True:
    #             for window in [2, 3, 5]:
    #                 add_param = add_param.rolling(window, axis=1).apply(angle, raw=True)
    #                 sub_column_name = f'macd_'+str(short)+'_angle_'+str(window)
    #                 params.append(sub_column_name)
    #                 add_param.columns = pd.MultiIndex.from_tuples([(col[0], sub_column_name) for col in add_param.columns])
    #                 df_out = df_out.join(add_param).sort_index(axis=1)
    #                 gc.collect()

    # gc.collect()
    return df, params